<a href="https://colab.research.google.com/github/VirtueZhao/Dive_into_Deep_Learning/blob/main/3_6_Implementation_of_Softmax_Regression_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import d2l
import torch
from IPython import display

In [4]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [5]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [6]:
X = torch.tensor([[1,2,3],[4,5,6]])
print(X)
print(X.sum(0))
print(X.sum(0).shape)
print("-----")
print(X.sum(0, keepdim=True))
print(X.sum(0, keepdim=True).shape)
print("-----")
print(X.sum(1))
print(X.sum(1).shape)
print("-----")
print(X.sum(1, keepdim=True))
print(X.sum(1, keepdim=True).shape)

tensor([[1, 2, 3],
        [4, 5, 6]])
tensor([5, 7, 9])
torch.Size([3])
-----
tensor([[5, 7, 9]])
torch.Size([1, 3])
-----
tensor([ 6, 15])
torch.Size([2])
-----
tensor([[ 6],
        [15]])
torch.Size([2, 1])


In [47]:
def softmax(X):
  X_exp = torch.exp(X)
  # print("X: ", X)
  # print("X_exp: ", X_exp)
  partition = X_exp.sum(1, keepdim=True)
  return X_exp / partition

In [8]:
X = torch.normal(0,1,(2,5))
X_prob = softmax(X)
print("X_prob: ", X_prob)
print("X_prob.sum(1): ", X_prob.sum(1,keepdim=True))

X:  tensor([[-0.9255, -0.5170, -0.8076, -0.6601, -1.7436],
        [-0.0203,  0.8960, -0.3690, -0.5898,  1.2350]])
X_exp:  tensor([[0.3963, 0.5963, 0.4459, 0.5168, 0.1749],
        [0.9799, 2.4499, 0.6914, 0.5545, 3.4382]])
X_prob:  tensor([[0.1861, 0.2799, 0.2093, 0.2426, 0.0821],
        [0.1208, 0.3019, 0.0852, 0.0683, 0.4237]])
X_prob.sum(1):  tensor([[1.],
        [1.]])


In [42]:
def net(X):
  return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [16]:
y = torch.tensor([0,2])
print("y: ",y)
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
print("y_hat: ",y_hat)
print("y_hat[[0,1],y]: ",y_hat[[0,1],y])
y_hat[[0,1],y] == y_hat[[0,1],[0,2]]

y:  tensor([0, 2])
y_hat:  tensor([[0.1000, 0.3000, 0.6000],
        [0.3000, 0.2000, 0.5000]])
y_hat[[0,1],y]:  tensor([0.1000, 0.5000])


tensor([True, True])

In [46]:
def cross_entropy(y_hat, y):
  # print("y_hat: ",y_hat)
  # print("y: ",y)
  # print("len(y_hat): ",len(y_hat))
  # print("range(len(y_hat)): ", range(len(y_hat)))

  # print(y_hat[range(len(y_hat)), y])
  return -torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [44]:
def accuracy(y_hat, y):
  # print("y_hat.shape: ", y_hat.shape)
  # print("y_hat.shape[1]: ", y_hat.shape[1])
  # print("y_hat: ", y_hat)

  if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
    t_hat = y_hat.argmax(axis=0)
    # print("t_hat: ", t_hat)
    y_hat = y_hat.argmax(axis=1)
    # print("y_hat: ", y_hat)
  
  # print("y type: ",type(y))
  # print("y_hat type: ",type(y_hat))

  cmp = y_hat.type(y.dtype) == y
  # print("cmp: ",cmp)
  # print("cmp.type(y.dtype): ",cmp.type(y.dtype))
  # print("cmp.type(y.dtype).sum(): ",cmp.type(y.dtype).sum())

  return float(cmp.type(y.dtype).sum())

y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
accuracy(y_hat, y) / len(y)

0.5

In [53]:
def evaluate_accuracy(net, data_iter):
  if isinstance(net, torch.nn.Module):
    net.eval()
  metric = Accumulator(2)
  for X, y in data_iter:
    metric.add(accuracy(net(X), y), y.numel())
    # print("metric[0]: ", metric[0])
    # print("metric[1]: ", metric[1])
  return metric[0] / metric[1]

In [62]:
class Accumulator:
  def __init__(self, n):
    self.data = [0.0] * n
    print("self.data: ", self.data)
  
  def add(self, *args):
    print("*args", args)
    self.data = [a + float(b) for a,b in zip(self.data, args)]
    print("zip(self.data, args): ", zip(self.data, args))
    # print("self.data: ", self.data)

  def reset(self):
    self.data = [0.0] * len(self.data)
  
  def __getitem__(self, idx):
    return self.data[idx]

In [63]:
evaluate_accuracy(net, test_iter)

self.data:  [0.0, 0.0]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


*args (45.0, 256)
zip(self.data, args):  <zip object at 0x7f599a779960>
*args (50.0, 256)
zip(self.data, args):  <zip object at 0x7f599a7f9730>
*args (33.0, 256)
zip(self.data, args):  <zip object at 0x7f599a628fa0>
*args (39.0, 256)
zip(self.data, args):  <zip object at 0x7f599a608370>
*args (41.0, 256)
zip(self.data, args):  <zip object at 0x7f599a62e5f0>
*args (41.0, 256)
zip(self.data, args):  <zip object at 0x7f599a62e5f0>
*args (46.0, 256)
zip(self.data, args):  <zip object at 0x7f599a5f51e0>
*args (41.0, 256)
zip(self.data, args):  <zip object at 0x7f599a6548c0>
*args (40.0, 256)
zip(self.data, args):  <zip object at 0x7f599a76cd20>
*args (36.0, 256)
zip(self.data, args):  <zip object at 0x7f599a608370>
*args (35.0, 256)
zip(self.data, args):  <zip object at 0x7f599a64e4b0>
*args (46.0, 256)
zip(self.data, args):  <zip object at 0x7f599a628fa0>
*args (35.0, 256)
zip(self.data, args):  <zip object at 0x7f599a5f5e60>
*args (44.0, 256)
zip(self.data, args):  <zip object at 0x7f599a

0.1603